<a href="https://colab.research.google.com/github/OmarAndujarL/Deep-Learning-Math514/blob/main/MAT514_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Data Preprocessing



## Download Required Packages

In [ ]:
#New tensorflow api
!pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 46.1 MB/s eta 0:00:00


In [ ]:
#Downlaod Package
!pip install yahoo_fin
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.7 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=795aa54f3c544b9dafa6914f89ba5b8adabae1e6960381a9e524a6cd4313ac5e
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24570 sha256=3c419944f06d204c726c65731721cbb2d128f076edb2c646ce20248f36eb2cf9
  Stored in directory: /root/.cache/pip/wheels/70/4b/f0/ea

## Loading Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
import scipy.stats as st
from yahoo_fin import options
import yfinance as yf
from datetime import datetime,timedelta
from scipy.stats import norm
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from scipy.optimize import minimize
print(tf.__version__)

2.14.0-dev20230509


## Data Access

In [ ]:
#Symbols
tick='^SPX'
index='^GSPC'
sp500 = yf.Ticker(index)
expirations=options.get_expiration_dates(tick)

In [ ]:
info = {}
for date in expirations:
  #Somehow there is an error with this date
    if date!='March 20, 2026':
        info[date] = options.get_options_chain(tick,date)

Calls=[]
Puts=[]
for date in expirations:
    if date!='March 20, 2026':
        Calls.append(info[date]['calls'][['Contract Name','Last Trade Date','Strike','Last Price']])
        Puts.append(info[date]['puts'][['Contract Name','Last Trade Date','Strike','Last Price']])

In [ ]:
Cs=pd.concat(Calls)
Cs=Cs.reset_index()
Cs=Cs.drop('index',axis=1)

Ps=pd.concat(Puts)
Ps=Ps.reset_index()
Ps=Ps.drop('index',axis=1)

#We only interested in the year-month-date
Cs['Date']=Cs['Last Trade Date'].map(lambda x:x[0:10])
Ps['Date']=Ps['Last Trade Date'].map(lambda x:x[0:10])

## Initial Look of the Dataset

Calls dataset

In [ ]:
Cs

,Contract Name,Last Trade Date,Strike,Last Price,Date
0,SPXW230509C02200000,2023-05-03 3:01PM EDT,2200.0,1937.60,2023-05-03
1,SPXW230509C02400000,2023-04-18 1:51PM EDT,2400.0,1761.10,2023-04-18
2,SPXW230509C02600000,2023-04-18 1:51PM EDT,2600.0,1561.62,2023-04-18
3,SPXW230509C03000000,2023-05-03 3:17PM EDT,3000.0,1117.20,2023-05-03
4,SPXW230509C03200000,2023-05-09 3:26PM EDT,3200.0,924.55,2023-05-09
...,...,...,...,...,...
5321,SPX271217C08800000,2022-12-30 1:35PM EDT,8800.0,27.88,2022-12-30
5322,SPX271217C09200000,2023-03-28 11:29AM EDT,9200.0,12.40,2023-03-28
5323,SPX271217C09600000,2023-04-28 11:44AM EDT,9600.0,13.42,2023-04-28
5324,SPX271217C10000000,2023-04-28 11:44AM EDT,10000.0,10.37,2023-04-28


Puts dataset

In [ ]:
Ps

,Contract Name,Last Trade Date,Strike,Last Price,Date
0,SPXW230509P01200000,2023-05-03 1:42PM EDT,1200.0,0.05,2023-05-03
1,SPXW230509P02200000,2023-04-18 1:51PM EDT,2200.0,0.05,2023-04-18
2,SPXW230509P02400000,2023-04-18 1:51PM EDT,2400.0,0.07,2023-04-18
3,SPXW230509P02600000,2023-04-26 4:04PM EDT,2600.0,0.05,2023-04-26
4,SPXW230509P02800000,2023-04-28 12:49PM EDT,2800.0,0.05,2023-04-28
...,...,...,...,...,...
6305,SPX271217P08000000,2023-04-25 12:25PM EDT,8000.0,2941.60,2023-04-25
6306,SPX271217P08400000,2023-04-12 9:30AM EDT,8400.0,3263.45,2023-04-12
6307,SPX271217P09600000,2023-03-10 11:51AM EDT,9600.0,4320.90,2023-03-10
6308,SPX271217P10000000,2023-03-10 11:51AM EDT,10000.0,4649.25,2023-03-10


## Calculate Moneyness

In [ ]:
#In this part, we will get the closing price of the underlying, which is S&P 500 index  from the earlist trade day of option to current time.
td=Cs['Last Trade Date'].sort_values()
td.iloc[0][0:10]

beg=td.iloc[0][0:10]
end=datetime.now()
end=end+timedelta(5)
end=datetime.strftime(end,'%Y-%m-%d')
Prices=sp500.history(start=beg,end=end)['Close']

In [ ]:
#Calculate Moneyness
#In this case, Moneyness is simply Strike/Price, K/S
M_c=[]
M_p=[]
for i in range(len(Cs)):
    if Cs.loc[i].Date in Prices.index:
        M_c.append(Cs.loc[i]['Strike']/Prices[(Cs.loc[i]['Date'])])
    else:
        M_c.append(np.nan)

for i in range(len(Ps)):
    if Ps.loc[i].Date in Prices.index:
        M_p.append(Ps.loc[i]['Strike']/Prices[(Ps.loc[i]['Date'])])
    else:
        M_p.append(np.nan)

In [ ]:
#Add the feature Moneyness to the two dataset
Cs['Moneyness']=M_c
Ps['Moneyness']=M_p

In [ ]:
#Check na, because there are observations that traded in weekends, but the market doesn't open during weekends
Cs.isna().sum(),Ps.isna().sum()

(Contract Name      0
 Last Trade Date    0
 Strike             0
 Last Price         0
 Date               0
 Moneyness          8
 dtype: int64,
 Contract Name      0
 Last Trade Date    0
 Strike             0
 Last Price         0
 Date               0
 Moneyness          4
 dtype: int64)

In [ ]:
#Drop nas
Cs=Cs.dropna()
Ps=Ps.dropna()

## Calculate Time to Maturity

In [ ]:
#calculate time to maturity
#Time to Maturity can be calculated as the T(expiration day) - t(current time)
tau_c=[];tau_p=[]
for i in range(len(Cs)):
    contract=Cs.iloc[i]['Contract Name']
    date=Cs.iloc[i].Date
    date= datetime.strptime(date,'%Y-%m-%d')
    if contract[0:4]=='SPXW':
        exp=contract[4:10]
        exp=datetime.strptime(exp, '%y%m%d')
        tau_c.append((exp-date).days/365)
    elif contract[0:3]=='SPX':
        exp=contract[3:9]
        exp=datetime.strptime(exp, '%y%m%d')
        tau_c.append((exp-date).days/365)
    else:
        tau_c.append(np.nan)
        
for i in range(len(Ps)):
    contract=Ps.iloc[i]['Contract Name']
    date=Ps.iloc[i].Date
    date= datetime.strptime(date,'%Y-%m-%d')
    if contract[0:4]=='SPXW':
        exp=contract[4:10]
        exp=datetime.strptime(exp, '%y%m%d')
        tau_p.append((exp-date).days/365)
    elif contract[0:3]=='SPX':
        exp=contract[3:9]
        exp=datetime.strptime(exp, '%y%m%d')
        tau_p.append((exp-date).days/365)
    else:
        tau_p.append(np.nan)

In [ ]:
#Add the feature Time to Maturity to the two datasets
Cs['Tau']=tau_c
Ps['Tau']=tau_p

<ipython-input-16-9f615404bf89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cs['Tau']=tau_c
<ipython-input-16-9f615404bf89>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ps['Tau']=tau_p


In [ ]:
#if there are weird date, we will get nas, but it seems that we have cleaned data
Cs.isna().sum(),Ps.isna().sum()

(Contract Name      0
 Last Trade Date    0
 Strike             0
 Last Price         0
 Date               0
 Moneyness          0
 Tau                0
 dtype: int64,
 Contract Name      0
 Last Trade Date    0
 Strike             0
 Last Price         0
 Date               0
 Moneyness          0
 Tau                0
 dtype: int64)

In [ ]:
#We will ned the feature current price for Black-scholes Model, so we add it to the dataframe
S0_C=[]
S0_P=[]
for i in range(len(Cs)):
  S0_C.append(Prices[Cs.iloc[i].Date])
for i in range(len(Ps)):
  S0_P.append(Prices[Ps.iloc[i].Date])
Cs['S0']=S0_C
Ps['S0']=S0_P

<ipython-input-18-187b255242bb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cs['S0']=S0_C
<ipython-input-18-187b255242bb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ps['S0']=S0_P


## Final Dataset for Black-Scholes Model

In [ ]:
#final dataset for Black-Scholes Model
Cs

,Contract Name,Last Trade Date,Strike,Last Price,Date,Moneyness,Tau,S0
0,SPXW230509C02200000,2023-05-03 3:01PM EDT,2200.0,1937.60,2023-05-03,0.537799,0.016438,4090.750000
1,SPXW230509C02400000,2023-04-18 1:51PM EDT,2400.0,1761.10,2023-04-18,0.577635,0.057534,4154.870117
2,SPXW230509C02600000,2023-04-18 1:51PM EDT,2600.0,1561.62,2023-04-18,0.625772,0.057534,4154.870117
3,SPXW230509C03000000,2023-05-03 3:17PM EDT,3000.0,1117.20,2023-05-03,0.733362,0.016438,4090.750000
4,SPXW230509C03200000,2023-05-09 3:26PM EDT,3200.0,924.55,2023-05-09,0.776856,0.000000,4119.169922
...,...,...,...,...,...,...,...,...
5321,SPX271217C08800000,2022-12-30 1:35PM EDT,8800.0,27.88,2022-12-30,2.291965,4.967123,3839.500000
5322,SPX271217C09200000,2023-03-28 11:29AM EDT,9200.0,12.40,2023-03-28,2.316639,4.726027,3971.270020
5323,SPX271217C09600000,2023-04-28 11:44AM EDT,9600.0,13.42,2023-04-28,2.302445,4.641096,4169.479980
5324,SPX271217C10000000,2023-04-28 11:44AM EDT,10000.0,10.37,2023-04-28,2.398381,4.641096,4169.479980


In [ ]:
Ps

,Contract Name,Last Trade Date,Strike,Last Price,Date,Moneyness,Tau,S0
0,SPXW230509P01200000,2023-05-03 1:42PM EDT,1200.0,0.05,2023-05-03,0.293345,0.016438,4090.750000
1,SPXW230509P02200000,2023-04-18 1:51PM EDT,2200.0,0.05,2023-04-18,0.529499,0.057534,4154.870117
2,SPXW230509P02400000,2023-04-18 1:51PM EDT,2400.0,0.07,2023-04-18,0.577635,0.057534,4154.870117
3,SPXW230509P02600000,2023-04-26 4:04PM EDT,2600.0,0.05,2023-04-26,0.641027,0.035616,4055.989990
4,SPXW230509P02800000,2023-04-28 12:49PM EDT,2800.0,0.05,2023-04-28,0.671547,0.030137,4169.479980
...,...,...,...,...,...,...,...,...
6305,SPX271217P08000000,2023-04-25 12:25PM EDT,8000.0,2941.60,2023-04-25,1.964815,4.649315,4071.629883
6306,SPX271217P08400000,2023-04-12 9:30AM EDT,8400.0,3263.45,2023-04-12,2.052811,4.684932,4091.949951
6307,SPX271217P09600000,2023-03-10 11:51AM EDT,9600.0,4320.90,2023-03-10,2.486023,4.775342,3861.590088
6308,SPX271217P10000000,2023-03-10 11:51AM EDT,10000.0,4649.25,2023-03-10,2.589607,4.775342,3861.590088


## Final Dataset for Machine Learning Models

In [ ]:
#final data set for Machine Learning Models
df_c=Cs[['Tau','Moneyness','Last Price']]
df_p=Ps[['Tau','Moneyness','Last Price']]
print("Data points:",len(df_c)+len(df_p))

Data points: 11624


Calls dataset pre-processed

In [ ]:
df_c

,Tau,Moneyness,Last Price
0,0.016438,0.537799,1937.60
1,0.057534,0.577635,1761.10
2,0.057534,0.625772,1561.62
3,0.016438,0.733362,1117.20
4,0.000000,0.776856,924.55
...,...,...,...
5321,4.967123,2.291965,27.88
5322,4.726027,2.316639,12.40
5323,4.641096,2.302445,13.42
5324,4.641096,2.398381,10.37


Puts dataset pre-processed

In [ ]:
df_p

,Tau,Moneyness,Last Price
0,0.016438,0.293345,0.05
1,0.057534,0.529499,0.05
2,0.057534,0.577635,0.07
3,0.035616,0.641027,0.05
4,0.030137,0.671547,0.05
...,...,...,...
6305,4.649315,1.964815,2941.60
6306,4.684932,2.052811,3263.45
6307,4.775342,2.486023,4320.90
6308,4.775342,2.589607,4649.25


# II.Models

## 2.1 Parametric Model

### Black-Scholes

In [ ]:
def BS_Price(CP,S0,K,sigma,T,r):
    d1=(log(S0/K)+(r-0.5*np.power(sigma,2))*T)/float((sigma*sqrt(T)))
    d2=d1-sigma*sqrt(T)
    if str(CP).lower()=='c' or str(CP).lower()=='1':
        value=S0*st.norm.cdf(d1)-K*np.exp(-r*T)*st.norm.cdf(d2)
    elif str(CP).lower()=='p' or str(CP).lower()=='0':
        value=-S0*st.norm.cdf(-d1)+K*np.exp(-r*T)*st.norm.cdf(-d2)
    return value

def Vector_BS(CP,param,obj):
  sigma,r=param
  BS=np.zeros([len(obj),1])
  for i in range(len(obj)):
    BS[i]=BS_Price(CP,obj.iloc[i]['S0'],obj.iloc[i]['Strike'],sigma,obj.iloc[i]['Tau'],r)
  return BS

def my_RMSE(yhat,y):
    return sqrt((yhat-y).T@(yhat-y))/len(yhat)
    
def my_MAE(yhat,y):
  dif=np.abs(y-yhat)
  return dif.sum()/len(yhat)

def my_R2(yhat,y): 
  dif=y-yhat
  sse=dif.T@dif
  sst=(y-y.mean()).T@(y-y.mean())
  return 1-float(sse/sst)



In [ ]:
#Calibration

def objective_Call(params,obj):
  sigma, r = params
  BS=np.zeros([len(obj),1])
  for i in range(len(obj)):
    BS[i]=BS_Price('C',obj.iloc[i]['S0'],obj.iloc[i]['Strike'],sigma,obj.iloc[i]['Tau'],r)
  mkt=obj['Last Price'].values.reshape([len(obj),1])
  return (BS-mkt).T@(BS-mkt)

def objective_Put(params,obj):
  sigma, r = params
  BS=np.zeros([len(obj),1])
  for i in range(len(obj)):
    BS[i]=BS_Price('P',obj.iloc[i]['S0'],obj.iloc[i]['Strike'],sigma,obj.iloc[i]['Tau'],r)
  mkt=obj['Last Price'].values.reshape([len(obj),1])
  return (BS-mkt).T@(BS-mkt)


sigma=0.2;r=0.05
param=sigma,r

#This part is very time consuming, takes about 20 minutes!!!
result_c = minimize(objective_Call,param,Cs)
result_p=minimize(objective_Put,param,Ps)

<ipython-input-24-fe342acb9495>:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1=(log(S0/K)+(r-0.5*np.power(sigma,2))*T)/float((sigma*sqrt(T)))
<ipython-input-24-fe342acb9495>:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1=(log(S0/K)+(r-0.5*np.power(sigma,2))*T)/float((sigma*sqrt(T)))
<ipython-input-24-fe342acb9495>:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1=(log(S0/K)+(r-0.5*np.power(sigma,2))*T)/float((sigma*sqrt(T)))


In [ ]:
#Calibrated Parameters
result_c.x,result_p.x

(array([0.17371637, 0.0367814 ]), array([0.19102673, 0.02862548]))

Technically, the fitted parameter by combing calls and puts  and fitted separately should be close. In practice, we find out that the the minimization algorithm is extremely time-consuming with the combined dataset, and the metrics are close to fitting separately. So we calibrate the parameters separately with calls and puts datasets. The calibrated parameters in both dataset are eonomically intuitive. The calls buyer may expect a smaller volatility than the puts buyer.

In [ ]:
#Calibrated BS Prices for calls:
BS_C=Vector_BS('C',result_c.x,Cs)
BS_P=Vector_BS('P',result_p.x,Ps)

#calculating metrics
BS_RMSE=my_RMSE(BS_C,Cs['Last Price'].values.reshape([len(Cs),1])),my_RMSE(BS_P,Ps['Last Price'].values.reshape([len(Ps),1]))
BS_MAE=my_MAE(BS_C,Cs['Last Price'].values.reshape([len(Cs),1])),my_MAE(BS_P,Ps['Last Price'].values.reshape([len(Ps),1]))
BS_R2=my_R2(BS_C,Cs['Last Price'].values.reshape([len(Cs),1])),my_R2(BS_P,Ps['Last Price'].values.reshape([len(Ps),1]))

<ipython-input-24-fe342acb9495>:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1=(log(S0/K)+(r-0.5*np.power(sigma,2))*T)/float((sigma*sqrt(T)))


In [ ]:
BS_metric=pd.DataFrame([BS_RMSE,BS_MAE,BS_R2],['RMSE','MAE','R^2'])
BS_metric.columns=['Call','Put']
BS_metric

,Call,Put
RMSE,0.458501,0.428386
MAE,19.231485,19.323843
R^2,0.997235,0.993404


## 2.2 Machine Learning Models

Set seed for reproducibility 

In [ ]:
seed = 42
tf.random.set_seed(seed)

### Working with call and put dataset simultaneously 

Train-test split

Here we have to decide which dataset to use. If we use "df_c" we'll be working with calls. If we choose "df_p" we'll be working with puts

In [ ]:
from sklearn.model_selection import train_test_split

model_df = df_c.copy()
model_df_p = df_p.copy()

#Call train/test

X = model_df[['Tau','Moneyness']].copy()
y = model_df['Last Price'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=42)

#Put train/test

X_p = model_df_p[['Tau','Moneyness']].copy()
y_p = model_df_p['Last Price'].copy()

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p, test_size = 0.15, random_state=42)

Normalization (Mean = 0, Sd = 1 for each column)

In [ ]:
# Call Normalization 
normalizer = preprocessing.Normalization() #creates the layer
normalizer.adapt(np.array(X_train)) #fit it to the data
pd.DataFrame(normalizer(X_train).numpy(), columns=X_train.columns).describe() #check the normalization

,Tau,Moneyness
count,4.520000e+03,4.520000e+03
mean,1.730117e-08,8.692783e-08
std,1.000111e+00,1.000111e+00
min,-6.023148e-01,-4.139539e+00
25%,-5.149385e-01,-2.539023e-01
50%,-3.123843e-01,6.614732e-02
75%,9.272398e-02,3.042968e-01
max,6.717437e+00,8.469946e+00


In [ ]:
# Put Normalization 
normalizer_p = preprocessing.Normalization() #creates the layer
normalizer_p.adapt(np.array(X_train_p)) #fit it to the data
pd.DataFrame(normalizer_p(X_train_p).numpy(), columns=X_train_p.columns).describe() #check the normalization

,Tau,Moneyness
count,5.360000e+03,5.360000e+03
mean,-6.476445e-08,-4.711436e-07
std,1.000093e+00,1.000093e+00
min,-5.715591e-01,-3.873671e+00
25%,-5.009274e-01,-3.429269e-01
50%,-3.555093e-01,1.520397e-01
75%,1.098286e-01,4.237202e-01
max,7.035884e+00,9.658786e+00


#### Linear Regression

First we create a linear regression model to estimate the prices given moneyness and tau

In [ ]:
def build_reg_model(norm):
  model = tf.keras.Sequential([
    norm,
    layers.Dense(units=1) #linear regression layer (units = 1)
  ])
  model.compile(
      optimizer=tf.optimizers.Adam(learning_rate=0.1),
      loss='mean_absolute_error'
      )

  return model

##### Call dataset

In [ ]:
reg_model_c = build_reg_model(normalizer)
reg_model_c.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 2)                 5         
 on)                                                             
                                                                 
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 8 (36.00 Byte)
Trainable params: 3 (12.00 Byte)
Non-trainable params: 5 (24.00 Byte)
_________________________________________________________________


In [ ]:
# Plot loss function of the trained model
import plotly.graph_objects as go

def plot_loss_plotly(history, tit):
  history_df = pd.DataFrame(history.history)
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=history_df.index.values, y=history_df['loss'],
                      mode='lines',
                      name='loss'))
  fig.add_trace(go.Scatter(x=history_df.index.values, y=history_df['val_loss'],
                      mode='lines',
                      name='val_loss'))
  fig.update_layout(title=tit,
                  xaxis_title='Epoch',
                   yaxis_title='Mean Absolute Error [Option Price]',
                    template='ggplot2',
                    autosize=False,
                    width=700,
                    height=500,)
  fig.show()

In [ ]:
#Model Training
%%time
history = reg_model_c.fit(
    X_train, y_train, 
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

CPU times: user 30.5 s, sys: 2.63 s, total: 33.1 s
Wall time: 31.8 s


In [ ]:
#Collect results for later
test_results = pd.DataFrame(data=[], columns=[])
test_results['linear_model'] = [reg_model_c.evaluate(
    X_test, y_test, verbose=0)]
test_results

,linear_model
0,141.777573


In [ ]:
plot_loss_plotly(history, 'Linear Regression Model Calls dataset')

In [ ]:
#Print Coefficients after training
Coefficients = pd.DataFrame(index=model_df.columns.to_list()[:-1])
Coefficients["Coefficients"] = reg_model_c.layers[1].kernel.numpy()
Coefficients

,Coefficients
Tau,242.359344
Moneyness,-636.254517


In [ ]:
test_predictions = reg_model_c.predict(X_test)
test_predictions.shape

25/25 [==============================] - 0s 1ms/step


(798, 1)

In [ ]:
y_true = np.array(y_test).reshape([len(y_test),1])
y_pred = np.array(test_predictions)
# Using 'auto'/'sum_over_batch_size' reduction type.
mse = tf.keras.losses.MeanSquaredError()
print(mse(y_true, y_pred).numpy())

103094.02


In [ ]:
rmse = np.sqrt(mse(y_true, y_pred).numpy())
rmse

321.08258

In [ ]:
R2 = my_R2(y_pred,y_true)
R2

0.7380014295782108

##### Puts dataset

In [ ]:
reg_model_p = build_reg_model(normalizer_p)
reg_model_p.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 2)                 5         
 tion)                                                           
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 8 (36.00 Byte)
Trainable params: 3 (12.00 Byte)
Non-trainable params: 5 (24.00 Byte)
_________________________________________________________________


In [ ]:
#Model Training
%%time
history = reg_model_p.fit(
    X_train_p, y_train_p, 
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

CPU times: user 34.8 s, sys: 2.67 s, total: 37.4 s
Wall time: 41.3 s


In [ ]:
#Collect results for later
test_results_p = pd.DataFrame(data=[], columns=[])
test_results_p['linear_model'] = [reg_model_p.evaluate(
    X_test_p, y_test_p, verbose=0)]

In [ ]:
plot_loss_plotly(history, 'Linear Regression Model Puts dataset')

Coefficients of the LR model

In [ ]:
#Print Coefficients after training
Coefficients = pd.DataFrame(index=model_df_p.columns.to_list()[:-1])
Coefficients["Coefficients"] = reg_model_p.layers[1].kernel.numpy()
Coefficients

,Coefficients
Tau,123.675011
Moneyness,111.773445


In [ ]:
test_predictions = reg_model_p.predict(X_test_p)
test_predictions.shape

30/30 [==============================] - 0s 2ms/step


(946, 1)

In [ ]:
y_true = np.array(y_test_p).reshape([len(y_test_p),1])
y_pred = np.array(test_predictions)
# Using 'auto'/'sum_over_batch_size' reduction type.
mse = tf.keras.losses.MeanSquaredError()
print(mse(y_true, y_pred).numpy())

133591.33


In [ ]:
rmse = np.sqrt(mse(y_true, y_pred).numpy())
rmse

365.50146

In [ ]:
R2 = my_R2(y_pred,y_true)
R2

0.4458504544967806

#### Deep Neural Network

In [ ]:
def build_dnn_model(norm, X_t):
  model = keras.Sequential([
    norm,
    layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)), #L2 regularization with a regularization strength of 0.01 to the weights of this layer.
    layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                    #loss='mean_absolute_percentage_error',
                    #loss='MeanSquaredLogarithmicError',
                  optimizer=tf.keras.optimizers.Adam(0.001),
                )
  model.build(X_t.shape)

  return model

##### Calls dataset

In [ ]:
DNN_model = build_dnn_model(normalizer, X_train)
DNN_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 2)                 5         
 on)                                                             
                                                                 
 dense_2 (Dense)             (None, 64)                192       
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                      

In [ ]:
%%time
#This part takes between 1-2 minutes to run. With Early stopping it takes 30s
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)  #Early stopping

history = DNN_model.fit(
    X_train, y_train,
    validation_split=0.2,
    verbose=1, epochs=100,
    callbacks=[early_stop])

Epoch 1/100
113/113 [==============================] - 3s 5ms/step - loss: 298.3108 - val_loss: 50.9225
Epoch 2/100
113/113 [==============================] - 0s 4ms/step - loss: 39.5536 - val_loss: 29.1392
Epoch 3/100
113/113 [==============================] - 0s 4ms/step - loss: 25.9449 - val_loss: 28.1289
Epoch 4/100
113/113 [==============================] - 0s 4ms/step - loss: 20.8627 - val_loss: 19.8771
Epoch 5/100
113/113 [==============================] - 0s 4ms/step - loss: 18.9383 - val_loss: 18.3622
Epoch 6/100
113/113 [==============================] - 0s 4ms/step - loss: 16.9260 - val_loss: 15.2942
Epoch 7/100
113/113 [==============================] - 0s 4ms/step - loss: 16.3088 - val_loss: 13.9307
Epoch 8/100
113/113 [==============================] - 0s 4ms/step - loss: 14.2717 - val_loss: 14.6814
Epoch 9/100
113/113 [==============================] - 0s 4ms/step - loss: 14.5077 - val_loss: 13.4982
Epoch 10/100
113/113 [==============================] - 0s 4ms/step - lo

In [ ]:
#Plot the loss
plot_loss_plotly(history, 'Deep Neural Network Model Calls dataset')

In [ ]:
#Collect Results
test_results['DNN_model'] = [DNN_model.evaluate(X_test, y_test, verbose=0)]
#Linear model Vs DNN
test_results.index = ["Mean Absolute Error [c]"]
test_results.T

,Mean Absolute Error [c]
linear_model,141.777573
DNN_model,12.406219


In [ ]:
test_predictions = DNN_model.predict(X_test)
test_predictions.shape

25/25 [==============================] - 0s 2ms/step


(798, 1)

In [ ]:
y_true = np.array(y_test).reshape([len(y_test),1])
y_pred = np.array(test_predictions)
# Using 'auto'/'sum_over_batch_size' reduction type.
mse = tf.keras.losses.MeanSquaredError()
mse(y_true, y_pred).numpy()

725.29846

In [ ]:
rmse = np.sqrt(mse(y_true, y_pred).numpy())
rmse

26.931366

R^2

In [ ]:
R2 = my_R2(y_pred,y_true)
R2

0.9981567588491678

##### Puts dataset

In [ ]:
DNN_model_p = build_dnn_model(normalizer_p, X_train_p)
DNN_model_p.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 2)                 5         
 tion)                                                           
                                                                 
 dense_8 (Dense)             (None, 64)                192       
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 64)                4160      
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                      

In [ ]:
%%time
#This part takes between 1-2 minutes to run
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)  #Early stopping
history = DNN_model_p.fit(
    X_train_p, y_train_p,
    validation_split=0.2,
    verbose=1, epochs=100,
    callbacks=[early_stop])

Epoch 1/100
134/134 [==============================] - 3s 5ms/step - loss: 112.3216 - val_loss: 52.3160
Epoch 2/100
134/134 [==============================] - 1s 4ms/step - loss: 37.2306 - val_loss: 29.2192
Epoch 3/100
134/134 [==============================] - 1s 5ms/step - loss: 23.2208 - val_loss: 17.5796
Epoch 4/100
134/134 [==============================] - 1s 4ms/step - loss: 16.1722 - val_loss: 14.4070
Epoch 5/100
134/134 [==============================] - 1s 4ms/step - loss: 12.7774 - val_loss: 11.3706
Epoch 6/100
134/134 [==============================] - 1s 5ms/step - loss: 11.3679 - val_loss: 13.9594
Epoch 7/100
134/134 [==============================] - 1s 4ms/step - loss: 10.6911 - val_loss: 11.2952
Epoch 8/100
134/134 [==============================] - 1s 4ms/step - loss: 10.9892 - val_loss: 11.2381
Epoch 9/100
134/134 [==============================] - 1s 4ms/step - loss: 10.4094 - val_loss: 11.6801
Epoch 10/100
134/134 [==============================] - 1s 4ms/step - lo

In [ ]:
#Plot the loss
plot_loss_plotly(history, 'Deep Neural Network Model Puts dataset')

Here we test the MAE of the linear regression against the deep learning model. We see that the DNN has a much better performance

In [ ]:
#Collect Results
test_results_p['DNN_model'] = [DNN_model_p.evaluate(X_test_p, y_test_p, verbose=0)]
#Linear model Vs DNN
test_results_p.index = ["Mean Absolute Error [p]"]
test_results_p.T

,Mean Absolute Error [p]
linear_model,110.717682
DNN_model,10.840566


In [ ]:
test_predictions = DNN_model_p.predict(X_test_p)
test_predictions.shape

30/30 [==============================] - 0s 2ms/step


(946, 1)

In [ ]:
y_true = np.array(y_test_p).reshape([len(y_test_p),1])
y_pred = np.array(test_predictions)
# Using 'auto'/'sum_over_batch_size' reduction type.
mse = tf.keras.losses.MeanSquaredError()
mse(y_true, y_pred).numpy()

1195.5726

In [ ]:
rmse = np.sqrt(mse(y_true, y_pred).numpy())
rmse

34.577053

In [ ]:
R2 = my_R2(y_pred,y_true)
R2

0.9950406504433714

#### Random Forest

In [ ]:
# Imports
# reference: https://www.youtube.com/watch?v=vZAjRCd03zk
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
#Train and test data sets
model_df_c = df_c.copy()
model_df_p = df_p.copy()

#Full train/test
X = model_df[['Tau','Moneyness']].copy()
y = model_df['Last Price'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=42)

#Put train/test
X_p = model_df_p[['Tau','Moneyness']].copy()
y_p = model_df_p['Last Price'].copy()
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p, test_size = 0.15, random_state=42)

#Call train/test
X_c = model_df_c[['Tau','Moneyness']].copy()
y_c = model_df_c['Last Price'].copy()
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c, y_c, test_size = 0.15, random_state=42)

In [ ]:
#Define regression models

reg = RandomForestRegressor()
reg.fit(X_train, y_train)

reg_p = RandomForestRegressor()
reg_p.fit(X_train_p, y_train_p)

reg_c = RandomForestRegressor()
reg_c.fit(X_train_c, y_train_c)

RandomForestRegressor()

In [ ]:
#Fit regression models
y_pred = reg.predict(X_test)
rf_results = pd.DataFrame(data=[], columns=[])
rf_results['y_pred'] = y_pred

y_pred_p = reg_p.predict(X_test_p)
rf_results_p = pd.DataFrame(data=[], columns=[])
rf_results_p['y_pred_p'] = y_pred_p

y_pred_c = reg_c.predict(X_test_c)
rf_results_c = pd.DataFrame(data=[], columns=[])
rf_results_c['y_pred_c'] = y_pred_c

In [ ]:
#Full data performance
print('Full data:')
print('R^2:', metrics.r2_score(y_test, y_pred))
print('Adjusted R^2:', 1-(1-metrics.r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

#Puts performance
print('Puts:')
print('R^2:', metrics.r2_score(y_test_p, y_pred_p))
print('Adjusted R^2:', 1-(1-metrics.r2_score(y_test_p, y_pred_p))*(len(y_test_p)-1)/(len(y_test_p)-X_test_p.shape[1]-1))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_p, y_pred_p))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_p, y_pred_p))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test_p, y_pred_p)))

#Calls performance
print('Calls:')
print('R^2:', metrics.r2_score(y_test_c, y_pred_c))
print('Adjusted R^2:', 1-(1-metrics.r2_score(y_test_c, y_pred_c))*(len(y_test_c)-1)/(len(y_test_c)-X_test_c.shape[1]-1))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_c, y_pred_c))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_c, y_pred_c))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test_c, y_pred_c)))

Full data:
R^2: 0.9985358209264799
Adjusted R^2: 0.9985321374571126
Mean Absolute Error: 10.784208270676686
Mean Squared Error: 576.1409876284703
RMSE: 24.002937062544454
Puts:
R^2: 0.9927349050569754
Adjusted R^2: 0.9927194965841376
Mean Absolute Error: 9.296991331923868
Mean Squared Error: 1751.4291632007964
RMSE: 41.85007960805805
Calls:
R^2: 0.9984119045069266
Adjusted R^2: 0.998407909298139
Mean Absolute Error: 10.98004135338347
Mean Squared Error: 624.900958069222
RMSE: 24.99801908290379


In [ ]:
#Full data plots
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, 
                         y=y_pred,
                         mode='markers'))
fig.update_layout(title="Predicted vs. Actual Prices - Full Data",
                  xaxis_title="Prices",
                  yaxis_title="Predicted prices",
                  template='ggplot2',
                  autosize=False,
                  width=700,
                  height=500,)
fig.show()

In [ ]:
#Put data plots
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test_p, 
                         y=y_pred_p,
                         mode='markers'))
fig.update_layout(title="Predicted vs. Actual Prices - Puts",
                  xaxis_title="Prices",
                  yaxis_title="Predicted prices",
                  template='ggplot2',
                  autosize=False,
                  width=700,
                  height=500,)
fig.show()

In [ ]:
#Call data plots
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test_c, 
                         y=y_pred_c,
                         mode='markers'))
fig.update_layout(title="Predicted vs. Actual Prices - Calls",
                  xaxis_title="Prices",
                  yaxis_title="Predicted prices",
                  template='ggplot2',
                  autosize=False,
                  width=700,
                  height=500,)
fig.show()

In [ ]:
#Full data residuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_pred, 
                         y=y_test-y_pred,
                         mode='markers'))
fig.update_layout(title="Predicted vs. Residuals - Full Data",
                  xaxis_title="Prices",
                  yaxis_title="Predicted prices",
                  template='ggplot2',
                  autosize=False,
                  width=700,
                  height=500,)
fig.show()

In [ ]:
#Full data residuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_pred_p, 
                         y=y_test_p-y_pred_p,
                         mode='markers'))
fig.update_layout(title="Predicted vs. Residuals - Puts",
                  xaxis_title="Prices",
                  yaxis_title="Predicted prices",
                  template='ggplot2',
                  autosize=False,
                  width=700,
                  height=500,)
fig.show()

In [ ]:
#Full data residuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_pred_c, 
                         y=y_test_c-y_pred_c,
                         mode='markers'))
fig.update_layout(title="Predicted vs. Residuals - Calls",
                  xaxis_title="Prices",
                  yaxis_title="Predicted prices",
                  template='ggplot2',
                  autosize=False,
                  width=700,
                  height=500,)
fig.show()

In [ ]:
# Cross validation - full data
scores = cross_val_score(reg, X_train, y_train, cv=100)
scores

array([0.99938608, 0.99601149, 0.99906298, 0.99926218, 0.99949137,
       0.99650436, 0.99883189, 0.99913199, 0.99929621, 0.99843961,
       0.99786177, 0.99594045, 0.99888175, 0.99887328, 0.99844324,
       0.99842716, 0.99819086, 0.99858912, 0.99926688, 0.99909213,
       0.99873909, 0.9989895 , 0.99880892, 0.99889872, 0.99942482,
       0.99580533, 0.99956216, 0.9986303 , 0.99678851, 0.99922688,
       0.99857244, 0.99866051, 0.99377244, 0.99817081, 0.99893755,
       0.99826293, 0.99870972, 0.99966777, 0.99879328, 0.9988277 ,
       0.99869993, 0.99961514, 0.9884181 , 0.9988126 , 0.99516487,
       0.99745319, 0.99786639, 0.99851309, 0.9984256 , 0.99949338,
       0.99936885, 0.99954634, 0.99850982, 0.99699821, 0.99963193,
       0.99920305, 0.99790599, 0.99883042, 0.99672769, 0.99380026,
       0.99917915, 0.998437  , 0.99868713, 0.99864581, 0.99774023,
       0.99906124, 0.99908115, 0.99963682, 0.99967955, 0.99904663,
       0.99857758, 0.99953993, 0.99917617, 0.99751328, 0.99954

In [ ]:
# Cross validation - puts
scores = cross_val_score(reg, X_train_p, y_train_p, cv=100)
scores

array([0.99454017, 0.9992343 , 0.99810666, 0.99968852, 0.98994461,
       0.99683738, 0.99913645, 0.99703169, 0.99903185, 0.9975738 ,
       0.98803268, 0.96293083, 0.99945821, 0.95021903, 0.99914409,
       0.99414035, 0.98961272, 0.97237575, 0.99770909, 0.98479492,
       0.9977972 , 0.99879857, 0.99905092, 0.99894435, 0.99576992,
       0.99645415, 0.99771893, 0.92294367, 0.9874337 , 0.99235272,
       0.99836006, 0.99076239, 0.99879106, 0.95530902, 0.99066436,
       0.99796048, 0.99902511, 0.99817779, 0.99010771, 0.99669498,
       0.99555334, 0.99147624, 0.99461837, 0.9862179 , 0.9975129 ,
       0.99768601, 0.98541248, 0.99745908, 0.99781502, 0.99139871,
       0.99925617, 0.99337513, 0.99924634, 0.99881288, 0.99656498,
       0.99555817, 0.99466925, 0.95811311, 0.99749172, 0.97448488,
       0.97561433, 0.99711687, 0.99883675, 0.99808902, 0.99825113,
       0.9946328 , 0.98267662, 0.99947113, 0.99676711, 0.9954812 ,
       0.99539638, 0.99936521, 0.98554229, 0.99831911, 0.99975

In [ ]:
# Cross validation - calls
scores = cross_val_score(reg, X_train_c, y_train_c, cv=100)
scores

array([0.99934791, 0.9962401 , 0.99881534, 0.99933785, 0.99948721,
       0.99627938, 0.99884144, 0.99910493, 0.99916786, 0.99859441,
       0.99781145, 0.99549053, 0.99909048, 0.99867921, 0.99841907,
       0.99834727, 0.998326  , 0.99848663, 0.99933164, 0.99910647,
       0.99887532, 0.99899518, 0.99884384, 0.99886559, 0.99954483,
       0.99608329, 0.99959811, 0.9984672 , 0.99613314, 0.99920466,
       0.99865085, 0.99857415, 0.99447337, 0.99834381, 0.99901876,
       0.99818282, 0.99867278, 0.99966374, 0.99895049, 0.99881439,
       0.99867658, 0.99962693, 0.98898629, 0.99878887, 0.9955279 ,
       0.99727646, 0.99800018, 0.99843691, 0.99845286, 0.99947326,
       0.99942438, 0.99963719, 0.99840898, 0.99701391, 0.99965386,
       0.99921413, 0.99793599, 0.99885028, 0.99653852, 0.99347166,
       0.99896775, 0.99829953, 0.99858481, 0.99837794, 0.99791182,
       0.99909864, 0.99897723, 0.99962141, 0.99970327, 0.99913922,
       0.99863629, 0.99953217, 0.9991553 , 0.9969699 , 0.99952

# III.Out-Sample Prediction

## Test Data

In [ ]:
tau=2/365;
S0=4130.11
C=[31.55,15.3,4.15]
C=np.array(C).reshape([len(C),1])

P=[7.61,14.65,32.93]
P=np.array(P).reshape([len(P),1])

K=[4110.0,4130.0,4160.0]
K=np.array(K).reshape([len(K),1])

Moneyness_c = K/S0

## Calibrated Black-Scholes

In [ ]:
C_os_bs=np.zeros([len(C),1])
P_os_bs=np.zeros([len(P),1])

sigma_c,r_c=result_c.x
sigma_p,r_p=result_p.x
for i in range(len(C)):
  C_os_bs[i]=BS_Price('C',S0,K[i],sigma_c,tau,r_c)
  P_os_bs[i]=BS_Price('P',S0,K[i],sigma_p,tau,r_p)



In [ ]:
#calculating metrics
BS_RMSE=my_RMSE(C_os_bs,C),my_RMSE(P_os_bs,P)
BS_MAE=my_MAE(C_os_bs,C),my_MAE(P_os_bs,P)
BS_R2=my_R2(C_os_bs,C),my_R2(P_os_bs,P)

### Performance

In [ ]:
BS_metric=pd.DataFrame([BS_RMSE,BS_MAE,BS_R2],['RMSE','MAE','R^2'])
BS_metric.columns=['Call(Out_sample)','Put(Out_Sample)']
BS_metric

,Call(Out_sample),Put(Out_Sample)
RMSE,2.893775,4.423085
MAE,4.568652,7.631901
R^2,0.801521,0.484575


## Linear regression 

In [ ]:
def compute_measures(y_true,y_pred):
  mae = tf.keras.losses.MeanAbsoluteError()
  mae1 = mae(y_true, y_pred).numpy()
  mse = tf.keras.losses.MeanSquaredError()
  rmse = np.sqrt(mse(y_true, y_pred).numpy())
  R2 = my_R2(y_pred,y_true) 

  return mae1,rmse,R2


In [ ]:
#For call
X_test = pd.DataFrame({'Tau':[2/365 for i in range(len(Moneyness_c))], 'Moneyness': Moneyness_c.reshape(len(Moneyness_c))})
test_predictions = reg_model_c.predict(X_test)
test_predictions.shape
y_true = C   #Real call prices
y_pred = np.array(test_predictions)
print('ytrue',y_true)
print()
print('ypred',y_pred)
mae1, rmse, R2 = compute_measures(y_true, y_pred)

#For put
test_predictions = reg_model_p.predict(X_test)
test_predictions.shape
y_true = P   #Real call prices
y_pred = np.array(test_predictions)
print('ytrue',y_true)
print()
print('ypred',y_pred)
mae1_p, rmse_p, R2_p = compute_measures(y_true, y_pred)

rmsel = [rmse,rmse_p]
mael = [mae1, mae1_p]
R2l = [R2, R2_p]

LR_metric = pd.DataFrame([rmsel, mael, R2l],['RMSE','MAE', 'R^2'])
LR_metric.columns=['Call(Out_sample)', 'Put(Out_Sample)']
LR_metric



1/1 [==============================] - 0s 18ms/step
ytrue [[31.55]
 [15.3 ]
 [ 4.15]]

ypred [[104.023544]
 [ 90.573944]
 [ 70.399475]]
1/1 [==============================] - 0s 19ms/step
ytrue [[ 7.61]
 [14.65]
 [32.93]]

ypred [[53.511448]
 [55.876648]
 [59.424458]]


,Call(Out_sample),Put(Out_Sample)
RMSE,71.431953,38.766544
MAE,71.332314,37.874187
R^2,-39.313325,-12.198001


## DNN Regression

In [ ]:
#For call
X_test = pd.DataFrame({'Tau':[2/365 for i in range(len(Moneyness_c))], 'Moneyness': Moneyness_c.reshape(len(Moneyness_c))})
test_predictions = DNN_model.predict(X_test)
test_predictions.shape
y_true = C   #Real call prices
y_pred = np.array(test_predictions)
print('ytrue',y_true)
print()
print('ypred',y_pred)
mae1, rmse, R2 = compute_measures(y_true, y_pred)

#For Put
test_predictions = DNN_model_p.predict(X_test)
test_predictions.shape
y_true = P   #Real call prices
y_pred = np.array(test_predictions)
print('ytrue',y_true)
print()
print('ypred',y_pred)
mae1_p, rmse_p, R2_p = compute_measures(y_true, y_pred)

rmsel = [rmse,rmse_p]
mael = [mae1, mae1_p]
R2l = [R2, R2_p]

DNN_metric = pd.DataFrame([rmsel, mael, R2l],['RMSE','MAE','R^2'])
DNN_metric.columns=['Call(Out_sample)', 'Put(Out_Sample)']
DNN_metric

1/1 [==============================] - 0s 19ms/step
ytrue [[31.55]
 [15.3 ]
 [ 4.15]]

ypred [[45.92736]
 [32.19661]
 [16.69007]]
1/1 [==============================] - 0s 33ms/step
ytrue [[ 7.61]
 [14.65]
 [32.93]]

ypred [[19.47534 ]
 [25.050926]
 [37.21514 ]]


,Call(Out_sample),Put(Out_Sample)
RMSE,14.713456,9.439774
MAE,14.604680,8.850469
R^2,-0.710381,0.217441


## Random Forest Regression

In [ ]:
#Fit regression models
rf_results_c = pd.DataFrame(data=[], columns=[])
rf_results_c['C'] = [31.55, 15.3, 4.15]
y_observed = np.array([31.55, 15.30, 4.15])

y_pred_c = reg_c.predict(X_test)
rf_results_c['y_pred_c'] = y_pred_c

rf_results_c

,C,y_pred_c
0,31.55,37.1469
1,15.30,24.5321
2,4.15,11.1456


In [ ]:
mae_c, rmse_c, r2_c = compute_measures(y_observed, y_pred_c)

In [ ]:
#Fit regression models
rf_results_p = pd.DataFrame(data=[], columns=[])
rf_results_p['P'] = [7.61, 14.65, 32.93]
y_observed = np.array([7.61, 14.65, 32.93])

y_pred_p = reg_p.predict(X_test)
rf_results_p['y_pred_p'] = y_pred_p

rf_results_p

,P,y_pred_p
0,7.61,13.4533
1,14.65,20.7353
2,32.93,37.1873


In [ ]:
mae_p, rmse_p, r2_p = compute_measures(y_observed, y_pred_p)

In [ ]:
r2 = [r2_c, r2_p]
mae = [mae_c, mae_p]
rmse = [rmse_c, rmse_p]

metrics = pd.DataFrame([r2, mae, rmse],['R^2', 'MAE', 'RMSE'])
metrics.columns=['Call', 'Put']
metrics

,Call,Put
R^2,0.564159,0.738590
MAE,7.274867,5.395300
RMSE,7.427323,5.455873
